In [75]:
# --- Import Libraries ---
import pandas as pd

# --- Load Raw Data ---
RAW_PATH = "../data/raw/food_inspections.csv"
df = pd.read_csv(RAW_PATH, low_memory=False)

# --- Initial Inspection ---
print("Shape:", df.shape)
print(df.head())


Shape: (296215, 17)
   Inspection ID                              DBA Name            AKA Name  \
0        2623075  HEALTHY WALA TIFFIN / FRIED JUNCTION      FRIED JUNCTION   
1        2623084                       KISHA'S KITCHEN     KISHA'S KITCHEN   
2        2623074                    BIG SAMS FOOD MART  BIG SAMS FOOD MART   
3        2623081                             FOOD TOWN           FOOD TOWN   
4        2623078                           JIMMY JOHNS         JIMMY JOHNS   

   License #  Facility Type             Risk               Address     City  \
0  2943658.0     Restaurant    Risk 1 (High)  6349 N CLAREMONT AVE  CHICAGO   
1  3006551.0     Restaurant    Risk 1 (High)        857 W 115TH ST  CHICAGO   
2  2817600.0  Grocery Store    Risk 1 (High)         133 E 75TH ST  CHICAGO   
3    31219.0  Grocery Store  Risk 2 (Medium)         935 E 79TH ST  CHICAGO   
4  3041206.0     Restaurant    Risk 1 (High)     3696 S ARCHER AVE  CHICAGO   

  State      Zip Inspection Date    

In [76]:
# --- Missing Values ---

# Count and percentage of missing values per column
missing = pd.DataFrame({
    "count_missing_values": df.isna().sum().sort_values(ascending=False),
    "missing_value_percentage": df.isna().mean().sort_values(ascending=False) * 100
})

print(missing)




                 count_missing_values  missing_value_percentage
Violations                      82449                 27.834175
Facility Type                    5240                  1.768985
AKA Name                         2413                  0.814611
Longitude                        1011                  0.341306
Location                         1011                  0.341306
Latitude                         1011                  0.341306
City                              159                  0.053677
Risk                               83                  0.028020
State                              58                  0.019580
Zip                                42                  0.014179
License #                          18                  0.006077
Inspection Type                     1                  0.000338
Inspection ID                       0                  0.000000
Address                             0                  0.000000
DBA Name                            0   

In [77]:
# --- Duplicates Check ---

# Count total duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Total duplicate rows: {duplicate_count}")



Total duplicate rows: 0


In [78]:
# --- Rename Columns ---

# Standardize column names: lowercase, replace '#' with 'number',
# replace spaces with underscores, and remove other special characters

df.columns = (
    df.columns.str.lower()
    .str.strip()
    .str.replace("#", "number")          # replace '#' first
    .str.replace(" ", "_")               # replace spaces
    .str.replace(r"[^a-z0-9_]", "", regex=True)  # cleanup other chars
)

print(df.columns)





Index(['inspection_id', 'dba_name', 'aka_name', 'license_number',
       'facility_type', 'risk', 'address', 'city', 'state', 'zip',
       'inspection_date', 'inspection_type', 'results', 'violations',
       'latitude', 'longitude', 'location'],
      dtype='object')


In [79]:
# --- Data Types Cleanup & Audit ---

# Ensure consistent types
df["license_number"] = df["license_number"].astype(str)
df["zip"] = df["zip"].astype(str)
df["inspection_id"] = df["inspection_id"].astype(str)
df["inspection_date"] = pd.to_datetime(df["inspection_date"], errors="coerce")

# Identify categorical and numerical columns
categorical_cols = df.select_dtypes(include=["object", "string", "category", "datetime64[ns]"]).columns.tolist()
numerical_cols = df.select_dtypes(include=["int64", "float64", "Int64"]).columns.tolist()

print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)




Categorical columns: ['inspection_id', 'dba_name', 'aka_name', 'license_number', 'facility_type', 'risk', 'address', 'city', 'state', 'zip', 'inspection_date', 'inspection_type', 'results', 'violations', 'location']
Numerical columns: ['latitude', 'longitude']


In [80]:
# --- Facility Type Cleanup ---

# Normalize text: lowercase → title case, fix "S" endings, remove stray apostrophes
df["facility_type"] = (
    df["facility_type"]
    .str.lower()
    .str.title()
    .str.replace(r"S\b", "'s", regex=True)
    .str.replace(r"'", "", regex=True)
)

# Check most common facility types
print(df["facility_type"].value_counts().head(10))





facility_type
Restaurant                         200211
Grocery Store                       36095
School                              18503
Childrens Services Facility          6939
Bakery                               4259
Daycare Above And Under 2 Years      3939
Daycare (2 - 6 Years)                3190
Long Term Care                       2291
Catering                             1835
Liquor                               1270
Name: count, dtype: int64


In [81]:
# --- Risk Cleanup ---

# Replace invalid value 'All' with NA
df.loc[df["risk"] == "All", "risk"] = pd.NA

# Check distribution after cleanup
print(df["risk"].value_counts(dropna=False))





risk
Risk 1 (High)      219081
Risk 2 (Medium)     53589
Risk 3 (Low)        23387
NaN                    83
<NA>                   75
Name: count, dtype: int64


In [82]:
# --- Results Cleanup ---

# Group inspection results into broader categories
results_mapping = {
    "Pass": "Pass-related",
    "Pass w/ Conditions": "Pass-related",
    "Fail": "Fail-related",
    "Fail w/ Conditions": "Fail-related",
    "Out of Business": "Other/Not Inspected",
    "No Entry": "Other/Not Inspected",
    "Not Ready": "Other/Not Inspected",
    "Business Not Located": "Other/Not Inspected"
}

df["results_grouped"] = df["results"].map(results_mapping)

# Check grouped results distribution
print(df["results_grouped"].value_counts(dropna=False))



results_grouped
Pass-related           197426
Fail-related            57337
Other/Not Inspected     41452
Name: count, dtype: int64


In [83]:
# --- Numerical Cleanup (Latitude/Longitude) ---

# Summary statistics for coordinates
print(df[["latitude", "longitude"]].describe())

# Drop rows with missing latitude/longitude
df = df.dropna(subset=["latitude", "longitude"])
print("After dropping missing coords:", df.shape)



            latitude      longitude
count  295204.000000  295204.000000
mean       41.880592     -87.676392
std         0.081087       0.058336
min        41.644670     -87.906874
25%        41.831302     -87.707598
50%        41.891797     -87.666524
75%        41.939753     -87.634955
max        42.021064     -87.525094
After dropping missing coords: (295204, 18)


In [84]:
# --- Bounding Box Filter ---

# Keep only rows within Chicago's latitude/longitude range
lat_min, lat_max = 41.6, 42.1
lon_min, lon_max = -88.0, -87.5

df = df[df["latitude"].between(lat_min, lat_max) & 
        df["longitude"].between(lon_min, lon_max)]

print("After bounding box filter:", df.shape)



After bounding box filter: (295204, 18)


In [85]:
# --- State Cleanup ---

df["state"].value_counts()

state
IL    295146
Name: count, dtype: int64

In [86]:
# --- City Cleanup ---

# Normalize to uppercase so we don't get 'Chicago', 'CHICAGO', 'chicago' separately
df["city"] = df["city"].str.upper()

# Quick look at counts
df["city"].value_counts().head(15)

# Map obvious typos back to CHICAGO
city_dict = {
    "CCHICAGO": "CHICAGO",
    "CHICAGOCHICAGO": "CHICAGO",
    "CHICAGOO": "CHICAGO",
    "CHICAGO.": "CHICAGO",
    "CHCHICAGO": "CHICAGO",
    "312CHICAGO": "CHICAGO",
    "CHICAGOI": "CHICAGO",
    "CHCICAGO": "CHICAGO",
    "CHICAGOC": "CHICAGO"
}
df["city"] = df["city"].replace(city_dict)

# Keep only Chicago rows (dropping suburbs + weird values)
before = df.shape[0]
df = df[df["city"] == "CHICAGO"]
after = df.shape[0]

print(f"Dropped {before - after} rows that weren’t Chicago.")
df.head()






Dropped 179 rows that weren’t Chicago.


,inspection_id,dba_name,aka_name,license_number,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location,results_grouped
0,2623075,HEALTHY WALA TIFFIN / FRIED JUNCTION,FRIED JUNCTION,2943658.0,Restaurant,Risk 1 (High),6349 N CLAREMONT AVE,CHICAGO,IL,60659.0,2025-08-29,Canvass Re-Inspection,No Entry,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.997380,-87.688504,"(41.99737973336337, -87.68850409776527)",Other/Not Inspected
1,2623084,KISHA'S KITCHEN,KISHA'S KITCHEN,3006551.0,Restaurant,Risk 1 (High),857 W 115TH ST,CHICAGO,IL,60643.0,2025-08-29,Complaint,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.684943,-87.644369,"(41.68494313726917, -87.644368901148)",Fail-related
2,2623074,BIG SAMS FOOD MART,BIG SAMS FOOD MART,2817600.0,Grocery Store,Risk 1 (High),133 E 75TH ST,CHICAGO,IL,60619.0,2025-08-29,Complaint,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.758208,-87.620088,"(41.75820844474927, -87.62008844011588)",Pass-related
3,2623081,FOOD TOWN,FOOD TOWN,31219.0,Grocery Store,Risk 2 (Medium),935 E 79TH ST,CHICAGO,IL,60619.0,2025-08-29,Canvass Re-Inspection,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.751218,-87.601401,"(41.75121793394575, -87.60140141021087)",Pass-related
4,2623078,JIMMY JOHNS,JIMMY JOHNS,3041206.0,Restaurant,Risk 1 (High),3696 S ARCHER AVE,CHICAGO,IL,60609.0,2025-08-29,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.826519,-87.684205,"(41.826518542923324, -87.68420498239537)",Pass-related


In [87]:
# --- ZIP Cleanup ---

# Convert to string, drop decimals (e.g. 60610.0 → 60610)
df["zip"] = df["zip"].astype(str).str.split(".", n=1, expand=True)[0]

# Check lengths (expect mostly 5 digits)
df["zip"].str.len().value_counts()

# Keep only 5-digit ZIPs
df = df[df["zip"].str.len() == 5]

# Keep only Chicago ZIPs
df = df[df["zip"].astype(int).between(60601, 60827)]

# ZIPs by count
print(df["zip"].value_counts().sort_index().tail(15))




zip
60649     3467
60651     4326
60652     2611
60653     2524
60654     6562
60655     1299
60656     1171
60657    10082
60659     6213
60660     4426
60661     3829
60666     3702
60706        3
60707     1716
60827      212
Name: count, dtype: int64


In [88]:
# --- Address Cleanup ---

# Strip whitespace and collapse multiple spaces into one
df["address"] = df["address"].str.strip().str.replace(r"\s{2,}", " ", regex=True)

# Check for missing addresses
print("Missing addresses:", df["address"].isna().sum())


Missing addresses: 0


In [89]:
df.head()

,inspection_id,dba_name,aka_name,license_number,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location,results_grouped
0,2623075,HEALTHY WALA TIFFIN / FRIED JUNCTION,FRIED JUNCTION,2943658.0,Restaurant,Risk 1 (High),6349 N CLAREMONT AVE,CHICAGO,IL,60659,2025-08-29,Canvass Re-Inspection,No Entry,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.997380,-87.688504,"(41.99737973336337, -87.68850409776527)",Other/Not Inspected
1,2623084,KISHA'S KITCHEN,KISHA'S KITCHEN,3006551.0,Restaurant,Risk 1 (High),857 W 115TH ST,CHICAGO,IL,60643,2025-08-29,Complaint,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.684943,-87.644369,"(41.68494313726917, -87.644368901148)",Fail-related
2,2623074,BIG SAMS FOOD MART,BIG SAMS FOOD MART,2817600.0,Grocery Store,Risk 1 (High),133 E 75TH ST,CHICAGO,IL,60619,2025-08-29,Complaint,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.758208,-87.620088,"(41.75820844474927, -87.62008844011588)",Pass-related
3,2623081,FOOD TOWN,FOOD TOWN,31219.0,Grocery Store,Risk 2 (Medium),935 E 79TH ST,CHICAGO,IL,60619,2025-08-29,Canvass Re-Inspection,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.751218,-87.601401,"(41.75121793394575, -87.60140141021087)",Pass-related
4,2623078,JIMMY JOHNS,JIMMY JOHNS,3041206.0,Restaurant,Risk 1 (High),3696 S ARCHER AVE,CHICAGO,IL,60609,2025-08-29,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.826519,-87.684205,"(41.826518542923324, -87.68420498239537)",Pass-related


In [90]:
# --- Drop Columns ---

# Remove 'location' (redundant since we have latitude/longitude)
df = df.drop(columns=["location"], errors="ignore")

# Quick check of the dataframe after drop
df.head()



,inspection_id,dba_name,aka_name,license_number,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,results_grouped
0,2623075,HEALTHY WALA TIFFIN / FRIED JUNCTION,FRIED JUNCTION,2943658.0,Restaurant,Risk 1 (High),6349 N CLAREMONT AVE,CHICAGO,IL,60659,2025-08-29,Canvass Re-Inspection,No Entry,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.997380,-87.688504,Other/Not Inspected
1,2623084,KISHA'S KITCHEN,KISHA'S KITCHEN,3006551.0,Restaurant,Risk 1 (High),857 W 115TH ST,CHICAGO,IL,60643,2025-08-29,Complaint,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.684943,-87.644369,Fail-related
2,2623074,BIG SAMS FOOD MART,BIG SAMS FOOD MART,2817600.0,Grocery Store,Risk 1 (High),133 E 75TH ST,CHICAGO,IL,60619,2025-08-29,Complaint,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.758208,-87.620088,Pass-related
3,2623081,FOOD TOWN,FOOD TOWN,31219.0,Grocery Store,Risk 2 (Medium),935 E 79TH ST,CHICAGO,IL,60619,2025-08-29,Canvass Re-Inspection,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.751218,-87.601401,Pass-related
4,2623078,JIMMY JOHNS,JIMMY JOHNS,3041206.0,Restaurant,Risk 1 (High),3696 S ARCHER AVE,CHICAGO,IL,60609,2025-08-29,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.826519,-87.684205,Pass-related


In [91]:
# --- Dates Cleanup ---

# Convert to datetime (invalid formats become NaT)
df["inspection_date"] = pd.to_datetime(df["inspection_date"], errors="coerce")

# Confirm type is datetime
print(df["inspection_date"].dtypes)

# Check overall date range
min_date = df["inspection_date"].min()
max_date = df["inspection_date"].max()
print(f"Date ranges from {min_date} to {max_date}")

# Count missing dates
print("Missing dates:", df["inspection_date"].isna().sum())




datetime64[ns]
Date ranges from 2010-01-04 00:00:00 to 2025-08-29 00:00:00
Missing dates: 0


In [92]:
df.head()

,inspection_id,dba_name,aka_name,license_number,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,results_grouped
0,2623075,HEALTHY WALA TIFFIN / FRIED JUNCTION,FRIED JUNCTION,2943658.0,Restaurant,Risk 1 (High),6349 N CLAREMONT AVE,CHICAGO,IL,60659,2025-08-29,Canvass Re-Inspection,No Entry,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.997380,-87.688504,Other/Not Inspected
1,2623084,KISHA'S KITCHEN,KISHA'S KITCHEN,3006551.0,Restaurant,Risk 1 (High),857 W 115TH ST,CHICAGO,IL,60643,2025-08-29,Complaint,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.684943,-87.644369,Fail-related
2,2623074,BIG SAMS FOOD MART,BIG SAMS FOOD MART,2817600.0,Grocery Store,Risk 1 (High),133 E 75TH ST,CHICAGO,IL,60619,2025-08-29,Complaint,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.758208,-87.620088,Pass-related
3,2623081,FOOD TOWN,FOOD TOWN,31219.0,Grocery Store,Risk 2 (Medium),935 E 79TH ST,CHICAGO,IL,60619,2025-08-29,Canvass Re-Inspection,Pass,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.751218,-87.601401,Pass-related
4,2623078,JIMMY JOHNS,JIMMY JOHNS,3041206.0,Restaurant,Risk 1 (High),3696 S ARCHER AVE,CHICAGO,IL,60609,2025-08-29,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.826519,-87.684205,Pass-related


In [93]:
# --- License Number Cleanup ---

# Convert to string, strip spaces, and remove decimal suffix (e.g., '2943658.0' → '2943658')
df["license_number"] = (
    df["license_number"]
    .astype(str)
    .str.strip()
    .str.split(".", n=1).str[0]
)

# Check distribution of license number lengths (e.g., 5, 6, 7 digits)
license_number_count = df["license_number"].dropna().str.len().value_counts()
print(license_number_count, "\n")

# Keep only rows with 7-digit license numbers (valid Chicago licenses)
df = df[df["license_number"].str.len() == 7]

# Quick preview after cleanup
print(df.head())


license_number
7    239371
5     46313
4      7661
3       791
1       785
2        58
6        43
Name: count, dtype: int64 

  inspection_id                              dba_name            aka_name  \
0       2623075  HEALTHY WALA TIFFIN / FRIED JUNCTION      FRIED JUNCTION   
1       2623084                       KISHA'S KITCHEN     KISHA'S KITCHEN   
2       2623074                    BIG SAMS FOOD MART  BIG SAMS FOOD MART   
4       2623078                           JIMMY JOHNS         JIMMY JOHNS   
5       2623073                        A LIL-ROC & RO      A LIL-ROC & RO   

  license_number  facility_type           risk               address     city  \
0        2943658     Restaurant  Risk 1 (High)  6349 N CLAREMONT AVE  CHICAGO   
1        3006551     Restaurant  Risk 1 (High)        857 W 115TH ST  CHICAGO   
2        2817600  Grocery Store  Risk 1 (High)         133 E 75TH ST  CHICAGO   
4        3041206     Restaurant  Risk 1 (High)     3696 S ARCHER AVE  CHICAGO   
5    

In [98]:
df.columns.str.strip()
df.head()

,inspection_id,dba_name,aka_name,license_number,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,results_grouped
0,2623075,HEALTHY WALA TIFFIN / FRIED JUNCTION,FRIED JUNCTION,2943658,Restaurant,Risk 1 (High),6349 N CLAREMONT AVE,CHICAGO,IL,60659,2025-08-29,Canvass Re-Inspection,No Entry,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",41.997380,-87.688504,Other/Not Inspected
1,2623084,KISHA'S KITCHEN,KISHA'S KITCHEN,3006551,Restaurant,Risk 1 (High),857 W 115TH ST,CHICAGO,IL,60643,2025-08-29,Complaint,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.684943,-87.644369,Fail-related
2,2623074,BIG SAMS FOOD MART,BIG SAMS FOOD MART,2817600,Grocery Store,Risk 1 (High),133 E 75TH ST,CHICAGO,IL,60619,2025-08-29,Complaint,Pass w/ Conditions,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.758208,-87.620088,Pass-related
4,2623078,JIMMY JOHNS,JIMMY JOHNS,3041206,Restaurant,Risk 1 (High),3696 S ARCHER AVE,CHICAGO,IL,60609,2025-08-29,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.826519,-87.684205,Pass-related
5,2623073,A LIL-ROC & RO,A LIL-ROC & RO,2918361,Restaurant,Risk 1 (High),2539 W 79TH ST,CHICAGO,IL,60652,2025-08-29,Canvass,Fail,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,41.749871,-87.686800,Fail-related
